In [7]:
# TESTING FOR NSE
# ***************

# Following to be uncommented on first run ...

from helper import get_connected
from ib_insync import *
util.startLoop()
ib = get_connected('nse', 'live') # close the connection and delete it final function

fspath = '../data/nse/'

util.logToFile(fspath+'_errors.log')  # create log file
with open(fspath+'_errors.log', 'w'): # clear the previous log
    pass

In [ ]:
%%time
# WEEKEND GENERATION PROCESS - UNDERLYINGS FIRST
# ______________________________________________

# testing pickle nses function

from nse_func import p_nses
df = p_nses(ib)
df

In [ ]:
%%time
# WEEKEND GENERATION PROCESS - OPTIONS SECOND
# ______________________________________________

# testing pickle nse options function

import pandas as pd
from os import listdir

from nse_func import p_nseopts

keep_pickles = True   # keep already pickled symbols

fspath = '../data/nse/'

df = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()

if keep_pickles:
    fs = listdir(fspath)
    optsList = [f[:-4] for f in fs if f[-3:] == 'pkl']
    dfr = df[~df.index.isin(optsList)] # remaining dfs (without existing pickles)
    [p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m) for u, p, z, m in zip(dfr.contract, dfr.undPrice, dfr.lot, dfr.margin)]
else:
    [p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m) for u, p, z, m in zip(df.contract, df.undPrice, df.lot, df.margin)]

In [ ]:
# EVERYDAY UPDATE PROCESS (FIRST PART)
# ___________________________________

# testing nse underlying update function

import pandas as pd
fspath = '../data/nse/'
dfu = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()
dfu

In [ ]:
dfu.loc[:, ['margin', 'undPrice']] = 0 # zerorize margin and undPrice to test
dfu

In [ ]:
%%time

from nse_func import upd_nses
upd_nses(ib, dfu)

In [ ]:
%%time

# EVERYDAY UPDATE PROCESS (SECOND PART - OPT PRICE)
# ________________________________________________

# Option price update
# Note: If the DataFrame given is a 'target' one, it will be every minute

import pandas as pd
fspath = '../data/nse/'

from os import listdir

from helper import upd_opt

blk = 50
fs = listdir(fspath)

# collect the options
optsList = [f for f in fs if f[-3:] == 'pkl']
dfopts = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)

# all options with price updated
df = upd_opt(ib, dfopts)

In [ ]:
df.head()

In [ ]:
%%time
# EVERYDAY UPDATE PROCESS (THIRD PART - REMAINING QUANTITIES PRICE)
# _________________________________________________________________

from nse_func import remqty_nse
dfrq = remqty_nse(ib) # remaining quantities df

In [19]:
# MANAGE NSE
# __________
import pandas as pd

from os import listdir

from nse_func import remqty_nse
from helper import grp_opts

# get remaining quantities
dfrq = remqty_nse(ib) # remaining quantities df

fspath = '../data/nse/'
minOptPrice = 0.15
minRom = 0.5
nLargest = 5

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

fs = listdir(fspath)

optsList = [f for f in fs if f[-3:] == 'pkl']

df1 = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

# filter for high probability and margin
df2 = df1[((df1.strike > df1.hi52) | 
           (df1.strike < df1.lo52)) & 
          (df1.optPrice > minOptPrice) & 
          (df1.rom > minRom)]

df2 = grp_opts(df2) # group the options
df2 = df2.assign(remqty=df2.symbol.map(dfrq.remqty.to_dict()))   # remaining quantities
df2 = df2[df2.remqty > 0].reset_index(drop=True) # remove blacklisted (remqty = 0)

In [32]:
premium = sum(df2.optPrice*df2.lotsize)

In [33]:
margin = sum(df2.optMargin)

In [34]:
premium/margin*365/4

1.3507433562308306